In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index

print("=> Assign each word with a unique index")
print(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index
print("=> Assign each tag with a unique index")
print(tag_to_ix)

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6



=> Assign each word with a unique index
{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}
=> Assign each tag with a unique index
{'DET': 0, 'NN': 1, 'V': 2}


In [3]:
################# Create the model #################
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [4]:
################# Train the model #################

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print()
    print("=> Scores before training of the tags affected to each word")
    print(tag_scores)


tensor([0, 1, 2, 3, 4])

=> Scores before training of the tags affected to each word
tensor([[-1.4600, -0.6581, -1.3866],
        [-1.4317, -0.6906, -1.3478],
        [-1.3247, -0.7654, -1.3131],
        [-1.4003, -0.7110, -1.3383],
        [-1.3666, -0.7435, -1.3108]])


In [6]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

    

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print()
    print('=> The sentence to analyze (first sentence of the Training data):')
    print(training_data[0][0])

    print()
    print("=> Training data: each word is assigned to a unique index:")
    print(word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!

    print()
    print("=> Scores after training of the tags affected to each word of the sentence to analyze:")
    print(tag_scores)

    
    indexList = []
    for tags in tag_scores:
        max = torch.max(tags).item()
        index = 0
        for tag in tags:  
            if tag.item() == max: 
                indexList.append(index)
            index +=1

    print(indexList)
    tagList = []
    for index in indexList: 
        tagList.append(list(tag_to_ix.keys())[index])
            
    print(tagList)



    

tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 0, 1])
tensor([5, 6, 7, 8])
tensor([1, 2, 0, 1])

=> The sentence to analyze (first sentence of the Training data):
['The', 'dog', 'ate', 'the', 'app